In [3]:
import re

import pandas as pd

df = pd.read_csv("../data/laptop-prices/Laptop+Prices.csv")

In [60]:
import re
from dash import Dash, html, dcc
import plotly.express as px

os_patterns = {
    "Windows": r"Windows",
    "macOS": r"macOS|OS X",
    "Linux": r"Linux",
    "Chrome OS": f"Chrome OS",
}

def extract_processor_series(model):
    match = re.search(r"(Core i[357]|Xeon|Pentinum|Celeron|Ryzen)", model, re.IGNORECASE)
    return match.group(1) if match else "Other"

def extract_graphics_card_series(model):
    match = re.search(r"(GeForce (GTX|RTX)|Radeon|(UHD|HD|Iris.*) Graphics)", model, re.IGNORECASE)
    return match.group(1) if match else "Other"

def categorize_os(os):
    for group, pattern in os_patterns.items():
        if re.search(pattern, os, re.IGNORECASE):
            return group
    return "Other"

df["CPU_series"] = df["CPU_model"].apply(extract_processor_series)
df["CPU_type"] = (
    df["CPU_company"].fillna("") + " " + df["CPU_series"].fillna("")
).str.strip()

df["GPU_series"] = df["GPU_model"].apply(extract_graphics_card_series)
df["GPU_type"] = (
    df["GPU_company"].fillna("") + " " + df["GPU_series"].fillna("")
).str.strip()

df["OS_group"] = df["OS"].apply(categorize_os)

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Not a Laptop Store", style={"textAlign": "center"}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                id="type",
                options=[
                    {"label": laptop_type, "value": laptop_type}
                    for laptop_type in df["TypeName"].unique()
                ],
                multi=False,
                placeholder="💻 Type...",
                style={"width": "200px"},
            ),
            dcc.Dropdown(
                id="brand",
                options=[
                    {"label": brand, "value": brand}
                    for brand in df["Company"].unique()
                ],
                multi=True,
                placeholder="✨ Brands...",
                style={"width": "200px"},
            ),
            dcc.Dropdown(
                id="os",
                options=[
                    {"label": os, "value": os}
                    for os in df["OS_group"].unique()
                ],
                multi=True,
                placeholder="🪟 Operating systems...",
                style={"width": "200px"},
            ),
            dcc.Dropdown(
                id="processor",
                options=[
                    {"label": processor, "value": processor}
                    for processor in df["CPU_type"].unique()
                ],
                multi=True,
                placeholder="⚙️ Processors...",
                style={"width": "200px"},
            ),
            dcc.Dropdown(
                id="graphics_card",
                options=[
                    {"label": graphics_card, "value": graphics_card}
                    for graphics_card in df["GPU_type"].unique()
                ],
                multi=True,
                placeholder="⚡ Graphics cards...",
                style={"width": "200px"},
            ),
        ], style={
            "display": "flex",
            "gap": "10px",
        }),
        html.Div([
            dcc.Checklist(
                id="touchscreen",
                options=[{"label": "📱 Has touchscreen?", "value": True}],
                value=[],
                inline=True,
            ),
        ], style={
            "display": "flex",
            "gap": "10px",
        }),
        dcc.RangeSlider(
            id="ram",
            min=0,
            max=df["Ram"].max(),
            step=4,
            value=[0, df["Ram"].max()],
            marks={
                i: f"{i}GB" if i in df["Ram"].unique() or i == 0 else ""
                for i in range(0, df["Ram"].max() + 1, 4)
            },
            tooltip={"placement": "top", "always_visible": True},
        ),
        dcc.RangeSlider(
            id="primary_storage",
            min=0,
            max=df["PrimaryStorage"].max(),
            step=128,
            value=[0, df["PrimaryStorage"].max()],
            marks={
                i: f"{i}GB" if i in df["PrimaryStorage"].unique() or i == 0 else ""
                for i in range(0, df["PrimaryStorage"].max() + 1, 128)
            },
            tooltip={"placement": "top", "always_visible": True},
        ),
        dcc.RangeSlider(
            id="price",
            min=0,
            max=df["Price_euros"].max(),
            step=1,
            value=[0, df["Price_euros"].max()],
            marks={
                i: f"€{i}"
                for i in range(0, int(df["Price_euros"].max()) + 1, 500)
            },
            tooltip={"placement": "top", "always_visible": True},
        ),
    ], style={
        "display": "flex",
        "flex-flow": "column",
        "gap": "20px",
    }),
])

app.run(jupyter_mode="inline", debug=True)